# Configuração, Inicialização e Carregamento de Dados

In [19]:
!pip install pyspark # nstala a biblioteca PySpark no ambiente de execução

In [20]:
import pyspark #  Importa a biblioteca principal do PySpark.
from pyspark.sql import SparkSession #  Importa a classe SparkSession, usada para iniciar a sessão Spark.
from pyspark.sql.functions import col # Importa a função col para referenciar colunas.
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA # VectorAssembler (para vetorizar features), MinMaxScaler (para normalização) e PCA (redução de dimensionalidade)
from pyspark.ml.clustering import KMeans # Importa o Algoritmo de Clustering: Importa o modelo K-Means.
from pyspark.ml.classification import DecisionTreeClassifier # Importa um Classificador: Importa a Árvore de Decisão
from pyspark.ml.evaluation import ClusteringEvaluator # Importa a classe para calcular métricas de clustering, como o Silhouette Score

In [21]:
from google.colab import drive # Importa o módulo drive do Google Colab.
drive.mount('/content/drive') # Monta o Google Drive no ambiente

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [23]:
pagamentos_pedido_preprocessados = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/output/pagamento_pedido_preprocessados_parquet') # Lê o DataFrame de Pagamentos Pré-processados
join_itens_pedido = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedido.parquet') # Lê o DataFrame de Itens de Pedido e Produtos (resultado de um join prévio)
produtos_preprocessados = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/output/produtos_preprocessados_parquet') # Lê o DataFrame de Produtos Pré-processados (com PCA/normalização de etapas anteriores)
avaliacoes_pedido = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/avaliacoes_pedido.csv',header=True, inferSchema=True) # Lê o DataFrame de Avaliações de Pedido
pedidos = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/pedidos.csv', header=True, inferSchema=True) # Lê o DataFrame de Pedidos

In [24]:
pagamentos_pedido_preprocessados.show(n=5, truncate=False) # Exibe as 5 primeiras linhas do DataFrame de pagamentos para inspeção
produtos_preprocessados.show(n=5, truncate=False) # Exibe as 5 primeiras linhas do DataFrame de produtos pré-processados
join_itens_pedido.show(n=5, truncate=False) # Exibe as 5 primeiras linhas do DataFrame de itens de pedido
avaliacoes_pedido.show(n=5, truncate=False) # Exibe as 5 primeiras linhas do DataFrame de avaliações de pedido

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333            |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664          |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664          |
|ba78997921bbcdc

# Preparação dos Dados para Clustering

In [25]:
itens_pedido_df = join_itens_pedido.na.drop() # Remove todas as linhas que contêm valores nulos (na.drop()) do DataFrame join_itens_pedido

# Define o transformador para agrupar as colunas de dimensão e peso (peso_produto_g, comprimento_produto_cm, etc.) em um único vetor chamado caracteristicas
montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g','comprimento_produto_cm','altura_produto_cm','largura_produto_cm'],
    outputCol='caracteristicas'
)

itens_pedido_df = montar_vetor.transform(itens_pedido_df) # Aplica o VectorAssembler, criando a coluna vetorial caracteristicas
itens_pedido_df.show(5) # Exibe as 5 primeiras linhas do DataFrame com a nova coluna caracteristicas

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------+--------------------+---------------

# Clustering K-Means (Sem Normalização)

In [26]:
# definir e treinar o modelo K-Means
Kmeans = KMeans(featuresCol='caracteristicas', k=3) # Define o algoritmo K-Means. A entrada é o vetor caracteristicas e o número de clusters (k) é definido como 3
modelo_kmeans = Kmeans.fit(itens_pedido_df) # Treina o modelo, encontrando os 3 centroides ideais.

In [27]:
predicao =modelo_kmeans.transform(itens_pedido_df) # Aplica o modelo treinado ao DataFrame para atribuir a cada produto um rótulo de cluster (prediction)
predicao.select('id_produto','prediction').show() # Exibe os IDs dos produtos e o cluster (0, 1 ou 2) ao qual foram atribuídos.

+--------------------+----------+
|          id_produto|prediction|
+--------------------+----------+
|4244733e06e7ecb49...|         0|
|ef92defde845ab845...|         0|
|557d850972a7d6f79...|         2|
|d63c1011f49d98b97...|         0|
|3f27ac8e699df3d30...|         0|
|5ed9eaf534f6936b5...|         0|
|553e0e7590d3116a0...|         0|
|5d7c23067ed3fc8c6...|         0|
|5a419dbf24a8c9718...|         0|
|21b1c2f67a9aafb5a...|         1|
|1c0c0093a48f13ba7...|         0|
|89321f94e35fc6d79...|         0|
|38afdf723b95d455b...|         0|
|672e757f331900b9d...|         0|
|28b4eced95a52d9c4...|         0|
|b10eba910a974df70...|         0|
|4089861a1bd4685da...|         0|
|fe59a1e006df3ac42...|         0|
|0b0172eb0fd18479d...|         0|
|0b0172eb0fd18479d...|         0|
+--------------------+----------+
only showing top 20 rows



In [28]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas', predictionCol='prediction', metricName='silhouette') # Cria o avaliador para calcular o Silhouette Score, que mede quão bem os objetos estão agrupados
silhouette = evaluator.evaluate(predicao) # Calcula o Silhouette Score para a rodada não-normalizada.
print(f"Silhouette Score: {silhouette}") # Imprime o score de Silhouette.

Silhouette Score: 0.91844438474722


# Normalização e Novo Clustering K-Means

In [29]:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado') # Define o transformador de normalização Min-Max.
modelo_scaler = scaler.fit(itens_pedido_df) # O fit calcula os valores mínimos e máximos das features
itens_pedido_normalizado_df= modelo_scaler.transform(itens_pedido_df) # Aplica a normalização, criando o vetor caracteristicas_normalizado

Kmeans = KMeans(featuresCol='caracteristicas_normalizado', k=3) # Define um novo modelo K-Means, usando o vetor normalizado como entrada
modelo_kmeans = Kmeans.fit(itens_pedido_normalizado_df) # Treina o modelo K-Means nos dados normalizados.

predicao_normalizada = modelo_kmeans.transform(itens_pedido_normalizado_df) # Aplica o novo modelo para obter os rótulos de cluster.
predicao_normalizada.select("id_produto","prediction").show() # Exibe os IDs dos produtos e seus novos clusters.

+--------------------+----------+
|          id_produto|prediction|
+--------------------+----------+
|4244733e06e7ecb49...|         2|
|ef92defde845ab845...|         2|
|557d850972a7d6f79...|         1|
|d63c1011f49d98b97...|         2|
|3f27ac8e699df3d30...|         2|
|5ed9eaf534f6936b5...|         2|
|553e0e7590d3116a0...|         2|
|5d7c23067ed3fc8c6...|         2|
|5a419dbf24a8c9718...|         0|
|21b1c2f67a9aafb5a...|         0|
|1c0c0093a48f13ba7...|         0|
|89321f94e35fc6d79...|         2|
|38afdf723b95d455b...|         2|
|672e757f331900b9d...|         2|
|28b4eced95a52d9c4...|         0|
|b10eba910a974df70...|         0|
|4089861a1bd4685da...|         2|
|fe59a1e006df3ac42...|         2|
|0b0172eb0fd18479d...|         2|
|0b0172eb0fd18479d...|         2|
+--------------------+----------+
only showing top 20 rows



In [30]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas_normalizado', predictionCol='prediction', metricName='silhouette') # Cria o avaliador para o novo conjunto de features.
silhouette = evaluator.evaluate(predicao_normalizada) # Calcula o Silhouette Score para a rodada normalizada
print(f"Silhouette Score: {silhouette}") # mprime o score de Silhouette, esperando um resultado superior ao anterior.

Silhouette Score: 0.6346504109754623


In [31]:
spark.stop() # Encerra a SparkSession